# Creates kerchunks from specified pattern or files on Azure Blobs

In [ ]:
#!pip install autopep8

import fsspec
import ujson
import xarray as xr
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr
from tqdm import tqdm

In [ ]:
import adlfs

print(adlfs.__version__)

In [ ]:
#If you want to see the available file systems
#fsspec.available_protocols()

## Needs to be improved... i.e. obtained from other credentials

In [ ]:
# needs to be improved
account_dict = dict(account_name = "<get this from the azure portal: storage account name>",
               account_key="<get this from the azure portal for this account name: key>")

In [ ]:
# Initiate fsspec filesystems for reading and writing
fs_read = fsspec.filesystem("abfs", **account_dict)

#fs_write = fsspec.filesystem("")
fs_write = fsspec.filesystem("file")

#!az login --use-device-code

In [ ]:
file_range=(1,3948) # (1,1000)

# Retrieve list of available files. Can take a long time
So instead used it to find out (by trial and error, could use a binary search here) to get the max number (3948) in this case

In [ ]:
blob_name = f"abfs://bay-delta-schism2-v58/eli/simulations/hindcast_clinic2/outputs/schout_0001_{file_range[1]-1}.nc"
print(f'Looking for last blob name: {blob_name}')
files_paths = fs_read.glob(blob_name)

assert len(files_paths) == 1 # else you have specified to far a range, if this fails reduce the range above

In [ ]:
# Here we prepend the prefix 'abfs://', which points to Azure Blobs.
#file_pattern = sorted(["abfs://" + f for f in files_paths])# faster if you already know the patterns expected.
file_pattern = [f'abfs://bay-delta-schism2-v58/eli/simulations/hindcast_clinic2/outputs/schout_0001_{i}.nc' 
                for i in range(*file_range)]

In [ ]:
file_pattern[0:3]

## Generate the zarr jsons for each file

In [ ]:
so_dict = dict(default_fill_cache=False, default_cache_type="first")
output_dir = "./hindcast2"

In [ ]:
import dask

## Start dask cluster

In [ ]:
import logging

from distributed import Client

client = Client(n_workers=8, silence_logs=logging.ERROR)
client

## Use the convenience function in kerchunk for dask
Discovered this later, still in progress as the above gives greater flexibility yet.

In [ ]:
account_dict

In [ ]:
from kerchunk.combine import auto_dask

auto_dask(file_pattern, single_driver=SingleHdf5ToZarr, single_kwargs = {}, mzz_kwargs = {}, 
          n_batches=8, remote_protocol='abfs', remote_options=account_dict,
          filename = f'hindcast2_0001.json')

## See other notebook to read using the combined files

[using combined json files to read](./azure_kerchunk_read_combined.ipynb)